# Random Forest para la clasificación

En este notebook vamos a construir un modelo de random forest para la clasificación en supervivientes o víctimas de los pasajeros del Titanic. Como vimos en la parte teórica el algoritmo de Random Forest soluciona mediante el muestreo de variables el problema de los árboles correlacionados generando una notoria mejora en la precisión de los modelos. Para la construcción de estos modelos emplearemos los siguientes módulos:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Cargamos como siempre los datos divididos en datos de entrenamiento y de validación:

In [ ]:
X_train = pd.read_csv('./data/xtrain_tit.csv')
X_test = pd.read_csv('./data/xtest_tit.csv')
y_train = pd.read_csv('./data/ytrain_tit.csv')
y_test = pd.read_csv('./data/ytest_tit.csv')

Verificamos que los datos se han cargado de manera correcta:

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
X_test.head()

In [ ]:
X_test.shape

Una vez comprobada la carga de los datos podemos proceder a la construcción del modelo de random forest para la clasificación.

## Construcción del modelo

Como siempre comenzamos instanciando el modelo:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier()

Una vez instanciado usamos los datos de entrenamiento para ajustar los parámetros:

In [ ]:
rf_classifier.fit(X_train, y_train)

Empleamos el conjunto de validación para evaluar la calidad del modelo:

In [ ]:
rf_classifier.score(X_train, y_train)

In [ ]:
rf_classifier.score(X_test, y_test)

Se presenta un ligero desfase entre la validación y el entrenamiento como ya es habitual. Podemos inferir que se produce algo de overfitting. Vamos a intentar ajustar los parámetros para reducirlo:

In [ ]:
rf_classifier_tuned = RandomForestClassifier(n_estimators = 20, max_depth = 3, random_state = 123)

In [ ]:
rf_classifier_tuned.fit(X_train, y_train)

In [ ]:
rf_classifier_tuned.score(X_train, y_train)

In [ ]:
rf_classifier_tuned.score(X_test, y_test)

Con los nuevos parámetros hemos paliado totalmente el overfitting (los datos en validación son mejores que en entrenamiento) y la precisión en validación se ha elevado ligeramente. Sobre este modelo mejorado calculamos la matriz de confusión y las métricas específicas de la clasificación:

In [ ]:
y_rf_pred = rf_classifier_tuned.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_rf_pred)
print(confusion_matrix)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_rf_pred))

Por último calculamos la curva ROC para poder evaluar el modeloy compararlo:

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, rf_classifier_tuned.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, rf_classifier_tuned.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Random Forest (Área bajo la curva = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Ratio de falsos positivos')
plt.ylabel('Ratio de verdaderos positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
#plt.savefig('Log_ROC') si descomentas esta línea puedes guardar la gráfica
plt.show()

Aunque hemos sacrificado parte de la explicabilidad aun podemos observar cuáles son las variables que han sido más relevantes:

In [ ]:
importancia_predictores = pd.DataFrame(
                            {'Variable predictora': X_train.columns,
                             'Importancia': rf_classifier_tuned.feature_importances_}
                            )
importancia_predictores.sort_values('Importancia', ascending=False)

De nuevo las variables más importantes son el sexo, la edad y si la persona va o no en tercera clase.